In [31]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [32]:
directory='PetImages'

train_data=tf.keras.utils.image_dataset_from_directory(
    directory,
    batch_size=15,
    image_size=(64,64),
    validation_split=0.3,
    subset='training',
    seed=42
)

validation_data=tf.keras.utils.image_dataset_from_directory(
    directory,
    validation_split=0.3,
    batch_size=15,
    image_size=(64,64),
    subset='validation',
    seed=42
)

Found 24991 files belonging to 2 classes.
Using 17494 files for training.
Found 24991 files belonging to 2 classes.
Using 7497 files for validation.


In [33]:
tune=tf.data.AUTOTUNE
train_data=train_data.cache().prefetch(buffer_size=tune)
validation_data=validation_data.cache().prefetch(buffer_size=tune)

In [37]:
base_model=tf.keras.applications.ResNet50(
    input_shape=(64,64,3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable=False

In [43]:
transfered_model=tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(64,64,3)),
    # tf.keras.layers.RandomFlip('horizontal'),
    # tf.keras.layers.RandomErasing(),
    # tf.keras.layers.RandomZoom(0.2),
    # tf.keras.layers.RandomBrightness(0.3),
    # tf.keras.layers.RandomContrast(0.2),
    # tf.keras.layers.RandomCrop(),
    # tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(200,activation='relu',kernel_regularizer=tf.keras.regularizers.L2),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Dense(50,activation='relu',kernel_regularizer=tf.keras.regularizers.L2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [44]:
transfered_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy','precision'])
transfered_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 2, 2, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_7      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 200)            │       409,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,997,713 (91.54 MB)

 Trainable params: 410,001 (1.56 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [45]:
transfered_model.fit(
    train_data,
    epochs=15,
    validation_data=validation_data,
    callbacks=[EarlyStopping(monitor='val_accuracy',restore_best_weights=True,patience=5,mode='max'),
               ModelCheckpoint('dog_cats_model_transfer.keras',monitor='val_loss',save_best_only=True,mode='min',verbose=1)
               ]
)

Epoch 1/15
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.7727 - loss: 1.9395 - precision: 0.7789
Epoch 1: val_loss improved from inf to 0.52677, saving model to dog_cats_model_transfer.keras
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 231s 194ms/step - accuracy: 0.7727 - loss: 1.9388 - precision: 0.7789 - val_accuracy: 0.8198 - val_loss: 0.5268 - val_precision: 0.8190
Epoch 2/15
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.8138 - loss: 0.5272 - precision: 0.8264
Epoch 2: val_loss improved from 0.52677 to 0.44387, saving model to dog_cats_model_transfer.keras
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 242s 208ms/step - accuracy: 0.8138 - loss: 0.5272 - precision: 0.8264 - val_accuracy: 0.8298 - val_loss: 0.4439 - val_precision: 0.8378
Epoch 3/15
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.8161 - loss: 0.4785 - precision: 0.8257
Epoch 3: val_loss improved from 0.44387 to 0.43758, saving model to dog_cats_model_transfer.keras
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 228s 196ms/step